In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium


In [ ]:
url = "https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$where=%20number_of_cyclist_killed%20%3E0%20AND%20crash_date%20%3E%20%222015-01-01%22%20&$limit=10000"
gdf = gpd.read_file(url)
trckrts = gpd.read_file("truckroutes/geo_export_9730ec6f-fb34-4095-86b0-507e5ace8052.shp")
bkrts = gpd.read_file("bikeroutes/geo_export_409825c9-67ff-4791-b9b7-41a11da81add.shp")

In [ ]:
def format (gdf):
    # drop rows w no geo (they have not lat or long either)
    gdf = gdf.loc[gdf.geometry.notna()]


    # select cols
    gdf = gdf[[ 'crash_date', 'crash_time',
         'on_street_name', 'off_street_name', 
       'number_of_cyclist_killed', 'number_of_motorist_killed', 'number_of_pedestrians_killed',
       'vehicle_type_code1', 'contributing_factor_vehicle_1',
       'vehicle_type_code2', 'contributing_factor_vehicle_2',
       'vehicle_type_code_3','contributing_factor_vehicle_3',
       'vehicle_type_code_4', 'contributing_factor_vehicle_4',
       'vehicle_type_code_5', 'contributing_factor_vehicle_5', 
       'geometry']]

    # put all in same format
    gdf.rename(columns={'vehicle_type_code1': 'vehicle_type_code_1', 'vehicle_type_code2': 'vehicle_type_code_2'}, inplace=True)

    # convert to ints
    gdf.number_of_cyclist_killed = gdf.number_of_cyclist_killed.astype(int)
    gdf.number_of_pedestrians_killed = gdf.number_of_pedestrians_killed.astype(int)
    gdf.number_of_motorist_killed = gdf.number_of_motorist_killed.astype(int)

    vehicles = pd.DataFrame()

    for i in range (1, 6):
        vehicle_code = 'vehicle_type_code_' + str(i)
        contributing_factor  =  'contributing_factor_vehicle_' + str(i)
        vehicles  = vehicles.append(gdf[['number_of_cyclist_killed', vehicle_code,
        contributing_factor, 'geometry']].rename(columns = {vehicle_code : 'vehicle_type_code',
        contributing_factor : 'contributing_factor_vehicle'}))
        
    return(vehicles)

In [ ]:
crashes = format(gdf)
crashes.head()

In [ ]:
print(crashes.shape)

crashes.dropna(subset=['geometry'])

In [ ]:
crashes.vehicle_type_code.unique()

In [ ]:
crashes = crashes[crashes['vehicle_type_code'].isin([
    'Box Truck', 'Garbage or Refuse','Tractor Truck Gasoline', 'Tow Truck / Wrecker','Tractor Truck Diesel',
    'Flat Bed', 'Flat Rack', 'Dump', 'Concrete Mixer', 'Tanker', 'LARGE COM VEH(6 OR MORE TIRES)'])]

In [ ]:
trckrts.to_crs('epsg:3857', inplace=True)
crashes.to_crs('epsg:3857', inplace=True)
bkrts.to_crs('epsg:3857', inplace=True)

In [ ]:
circles = crashes.copy()
circles.geometry = circles.geometry.buffer(100)

fig, ax = plt.subplots(figsize=(40, 40))
# cx.add_basemap(ax, source=cx.providers.CartoDB.Voyager)
# trckrts.plot(ax=ax, color='blue', zorder=1)
bkrts.plot(ax=ax, color='green')
circles.plot(ax=ax, color='red')

ax.set_axis_off()

In [ ]:
trckcrashes = gpd.sjoin(crashes, trckrts, how='left', predicate='crosses')
notrckcrashes = trckcrashes.loc[trckcrashes['index_right'].isna()]
trckcrashes = trckcrashes.loc[trckcrashes['index_right'].notna()]
trckcrashes

In [ ]:



m = bkrts.explore(color='green')
m = trckrts.explore(m=m, color='blue')
m = crashes.explore(m=m, color='red')
# folium.LayerControl().add_to(m)

m


In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))
# cx.add_basemap(ax, source=cx.providers.CartoDB.Voyager)
trckcrashes.plot(ax=ax, color='red', zorder=1)
notrckcrashes.plot(ax=ax, color='blue')

ax.set_axis_off()